# Cross Validation

1. Reservar 1/k de los datos como muestra reservada.
2. Entrenar el modelo con los datos restantes
3. Aplicar (puntuar) el modelo a la retención de 1/k y registrar las métricas de evaluación del modelo.
4. Restaurar el primer 1/k de los datos y reservar el siguiente 1/k (excluyendo los registros que se seleccionaron la primera vez)
5. Repetir los pasos 2 y 3
6. Repetir hasta que cada registro se haya utilizado en la parte reservada
7. Promediar o combinar las métricas de evaluación del modelo.

In [11]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
#import statsmodels.api as sm

In [12]:
k = 10

In [13]:
data = pd.read_csv("../Datasets/house_sales.csv",delimiter="\t")

data

,DocumentDate,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
1,2014-09-16,280000,1000102,Multiplex,2014-09-01,405100,0.930836,300805.0,2,9373,...,3.00,6,7,1991,0,0,70000,229000,98002,False
2,2006-06-16,1000000,1200013,Single Family,2006-06-01,404400,0.929228,1076162.0,1,20156,...,3.75,4,10,2005,0,0,203000,590000,98166,True
3,2007-01-29,745000,1200019,Single Family,2007-01-01,425600,0.977941,761805.0,1,26036,...,1.75,4,8,1947,0,0,183000,275000,98166,False
4,2008-02-25,425000,2800016,Single Family,2008-02-01,418400,0.961397,442065.0,1,8618,...,3.75,5,7,1966,0,0,104000,229000,98168,False
5,2013-03-29,240000,2800024,Single Family,2013-03-01,351600,0.807904,297065.0,1,8620,...,1.75,4,7,1948,0,0,104000,205000,98168,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27057,2011-04-08,325000,9842300710,Single Family,2011-04-01,318700,0.732307,443803.0,1,5468,...,1.75,3,7,1951,0,0,201000,172000,98126,False
27058,2007-09-28,1580000,9845500010,Single Family,2007-09-01,433500,0.996094,1586196.0,1,23914,...,4.50,4,11,2000,0,1,703000,951000,98040,False
27061,2012-07-09,165000,9899200010,Single Family,2012-07-01,325300,0.747472,220744.0,1,11170,...,1.00,4,6,1971,0,0,92000,130000,98055,False
27062,2006-05-26,315000,9900000355,Single Family,2006-05-01,400600,0.920496,342207.0,1,6223,...,2.00,3,7,1939,0,0,103000,212000,98166,False


In [14]:
slices = np.array_split(data,10)
slices

[     DocumentDate  SalePrice  PropertyID   PropertyType          ym  zhvi_px  \
 1      2014-09-16     280000     1000102      Multiplex  2014-09-01   405100   
 2      2006-06-16    1000000     1200013  Single Family  2006-06-01   404400   
 3      2007-01-29     745000     1200019  Single Family  2007-01-01   425600   
 4      2008-02-25     425000     2800016  Single Family  2008-02-01   418400   
 5      2013-03-29     240000     2800024  Single Family  2013-03-01   351600   
 ...           ...        ...         ...            ...         ...      ...   
 2775   2009-07-29     198000  1099600540  Single Family  2009-07-01   354400   
 2776   2015-03-26     160000  1099600620  Single Family  2015-03-01   423200   
 2777   2007-01-30     263500  1099610260  Single Family  2007-01-01   425600   
 2778   2006-11-20     257000  1099610330  Single Family  2006-11-01   421200   
 2779   2006-10-02     365000  1099611080  Single Family  2006-10-01   418300   
 
       zhvi_idx  AdjSalePr

In [15]:
slices[0]

,DocumentDate,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
1,2014-09-16,280000,1000102,Multiplex,2014-09-01,405100,0.930836,300805.0,2,9373,...,3.00,6,7,1991,0,0,70000,229000,98002,False
2,2006-06-16,1000000,1200013,Single Family,2006-06-01,404400,0.929228,1076162.0,1,20156,...,3.75,4,10,2005,0,0,203000,590000,98166,True
3,2007-01-29,745000,1200019,Single Family,2007-01-01,425600,0.977941,761805.0,1,26036,...,1.75,4,8,1947,0,0,183000,275000,98166,False
4,2008-02-25,425000,2800016,Single Family,2008-02-01,418400,0.961397,442065.0,1,8618,...,3.75,5,7,1966,0,0,104000,229000,98168,False
5,2013-03-29,240000,2800024,Single Family,2013-03-01,351600,0.807904,297065.0,1,8620,...,1.75,4,7,1948,0,0,104000,205000,98168,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,2009-07-29,198000,1099600540,Single Family,2009-07-01,354400,0.814338,243142.0,1,6600,...,1.00,2,7,1970,0,0,58000,116000,98023,False
2776,2015-03-26,160000,1099600620,Single Family,2015-03-01,423200,0.972426,164537.0,1,6497,...,1.50,3,7,1970,0,0,58000,122000,98023,False
2777,2007-01-30,263500,1099610260,Single Family,2007-01-01,425600,0.977941,269444.0,1,12705,...,1.75,4,7,1971,0,0,58000,131000,98023,False
2778,2006-11-20,257000,1099610330,Single Family,2006-11-01,421200,0.967831,265542.0,1,7290,...,1.00,3,7,1976,0,0,58000,139000,98023,False


In [16]:
outcome = 'AdjSalePrice'
predictors = ['zhvi_px',
 'NbrLivingUnits',
 'SqFtLot',
 'SqFtTotLiving',
 'SqFtFinBasement',
 'Bedrooms',
 'BldgGrade',
 'YrRenovated',
 'LandVal',
 'ImpsVal',
 'ZipCode',
 'NewConstruction']

In [17]:
rsquared_results = []
f_statistic_results = []
p_values_results = []

for i in range(len(slices)):
 reserved_data = slices[i]
 left_side = slices[:i]
 right_side = slices[i+1:]
 remaining_data = pd.concat(left_side+right_side) # Reto: Unir dos partes

 #Entrenamiento
 model = LinearRegression()
 model.fit(remaining_data[predictors],remaining_data[outcome])

 rsquared = model.score(reserved_data[predictors],reserved_data[outcome])
 f_statistic,p_values = f_regression(reserved_data[predictors],reserved_data[outcome])

 rsquared_results.append(rsquared)
 f_statistic_results.append(f_statistic)
 p_values_results.append(p_values)

In [18]:
rsquared_results # Promediar los resultados
mean_r2 = sum(rsquared_results)/len(rsquared_results)
mean_r2

0.8686254566193533